## 1. Primeros pasos con SparkSQL

Spark SQL es un módulo de Spark para el procesamiento de datos estructurados. A diferencia de la API de Spark RDD, las interfaces proporcionadas por Spark SQL brindan a Spark más información sobre la estructura tanto de los datos, como de los cálculos que se están realizando.

Internamente, Spark SQL usa esta información extra para realizar optimizaciones adicionales. Hay varias formas de interactuar con Spark SQL, incluidas SQL y la Dataframe API. Al calcular un resultado, se utiliza el mismo motor de ejecución, independientemente de qué API / lenguaje esté utilizando para expresar el cálculo. Esta unificación significa que los desarrolladores pueden alternar fácilmente entre diferentes API en función de cuál proporciona la forma más natural de expresar unas transformaciones determinadas.

SparkSQL se sustenta sobre el concepto de Dataframe, la cual es una colección de datos tabular, que al igual que los RDDs, se encuentra distribuida en los diferentes nodos de un cluster.

### 1.1. SQLContext

El punto de entrada a las funcionalidades de SparkSQL es la clase SparkSesion. Para crear una SparkSession básica:

In [2]:
# Importar librerías y dependencias
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .master("local[*]")\
                    .config("spark.app.name", "myApp")\
                    .getOrCreate()

In [8]:
spark

In [9]:
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1637406307040'),
 ('spark.rdd.compress', 'True'),
 ('spark.sql.warehouse.dir',
  'file:/home/gmachin/git/project_exploring_spark/spark-warehouse'),
 ('spark.driver.host', '192.168.1.43'),
 ('spark.driver.port', '44009'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1637406308066'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

### 1.2. Dataframe

In [10]:
import random

random.seed(7)
ids = range(5)
species = random.choices(['capibara', 'naked mole rat', 'axolotl', 'flying fox'], k=5)

data = list(zip(ids, species))
data

[(0, 'naked mole rat'),
 (1, 'capibara'),
 (2, 'axolotl'),
 (3, 'capibara'),
 (4, 'axolotl')]

In [11]:
df = spark.createDataFrame(data)
df

DataFrame[_1: bigint, _2: string]

In [12]:
df.collect()

[Row(_1=0, _2='naked mole rat'),
 Row(_1=1, _2='capibara'),
 Row(_1=2, _2='axolotl'),
 Row(_1=3, _2='capibara'),
 Row(_1=4, _2='axolotl')]

In [13]:
df.show()

+---+--------------+
| _1|            _2|
+---+--------------+
|  0|naked mole rat|
|  1|      capibara|
|  2|       axolotl|
|  3|      capibara|
|  4|       axolotl|
+---+--------------+



## 2. Bibliografía

- https://spark.apache.org/docs/latest/rdd-programming-guide.html
- https://spark.apache.org/docs/latest/sql-programming-guide.html
- Libro: Introducción a Apache Spark 